In [1]:
import logging

DEFAULT_FORMAT = "%(asctime)s [%(process)d] [%(name)s] [%(levelname)s]: %(message)s"
logging.basicConfig(level=logging.WARNING, format=DEFAULT_FORMAT)

SEARXNG_ENDPOINT = "http://localhost:8080"

In [2]:
from gba.client import MistralInstruct, Llama3Instruct, LlamaCppClient
from sentence_transformers import SentenceTransformer, CrossEncoder

# Proxy for 8-bit finetuned Mistral-7B-v0.1 planner
mistral_finetuned = MistralInstruct(
    llm=LlamaCppClient(url="http://localhost:8082/completion", temperature=-1),
)

# Proxy for 4-bit quantized Llama-3-8B-Instruct
llama3_instruct = Llama3Instruct(
    llm=LlamaCppClient(url="http://localhost:8084/completion", temperature=-1),
)

# Proxy for 8-bit quantized NexusRaven-V2-13B
nexus_raven = LlamaCppClient(url="http://localhost:8089/completion", temperature=-1)

embedding_model = SentenceTransformer(
    "mixedbread-ai/mxbai-embed-large-v1",
    device="cuda:0",
)

rerank_model = CrossEncoder(
    "mixedbread-ai/mxbai-rerank-large-v1",
    device="cuda:1",
)

/home/glados/miniconda3/envs/grammar-based-agents/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/glados/miniconda3/envs/grammar-based-agents/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/glados/miniconda3/envs/grammar-based-agents/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from pathlib import Path
import logging

from datasets import load_from_disk
from gba.client import Llama3Instruct, LlamaCppClient
from gba.utils import Scratchpad
from gba.tools.search import SearchInternetTool, SearchWikipediaTool, ContentExtractor

from gba.tools.search.search_wikipedia import SearchWikipediaTool
from gba.tools.search.extract import ContentExtractor

search_wikipedia_tool = SearchWikipediaTool(
    llm_model=llama3_instruct,
    embedding_model=embedding_model,
    rerank_model=rerank_model,                    
    top_k_nodes=10,
    top_k_related_documents=1,
    top_k_related_nodes=3,
    extractor=ContentExtractor(model=llama3_instruct),
)

search_internet_tool = SearchInternetTool(
    llm_model=llama3_instruct,
    rerank_model=rerank_model,
    searxng_endpoint=SEARXNG_ENDPOINT,
    fetch_webpage_timeout=5.0,
    top_k_documents=3,
    top_k_nodes_per_document=5,
    top_k_snippets=None,
    extractor=ContentExtractor(model=llama3_instruct),
)

2024-05-24 18:36:24,923 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Loading document mapping...
2024-05-24 18:36:25,087 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Loading binary search index...


In [4]:
search_wikipedia_tool.run(
    task="what are the physical characteristics of the Halicreas minimum",
    request="",
    scratchpad=None,
)

2024-05-24 18:36:34,352 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'What are the physical characteristics of Halicreas minimum?'


'The physical characteristics of Halicreas minimum include an umbrella 30–40 mm wide, thick, disk-like with a small apical projection, and 8 clusters of gelatinous papillae above the margin.'

In [5]:
search_internet_tool.run(
    task="Leo DiCaprio's current girlfriend's name and age", 
    scratchpad=None,
    request=""
)

2024-05-24 18:36:43,280 [1091672] [gba.tools.search.search_internet] [WARNING]: Searching the internet for query 'What is Leo DiCaprio's current girlfriend's name and age?'


"Leo DiCaprio's current girlfriend is Vittoria Ceretti, who is 25 years old."

In [6]:
from gba.agent import Agent
from gba.client import ChatClient
from gba.planner import FineTunedPlanner
from gba.search import SearchEngine
from gba.store import DocumentStore
from gba.summary import ResultSummarizer
from gba.tools import *

from example_docs import DOCUMENTS
from example_funcs import create_event

store = DocumentStore(path=".chroma")
engine = SearchEngine(store=store)

if store.count() == 0:
    for i, document in enumerate(DOCUMENTS):
        store.add(identifier=str(i), document=document)

summarizer = ResultSummarizer(model=llama3_instruct)

tools = [
    AskTool(),
    search_wikipedia_tool,
    search_internet_tool,
    CalculateTool(model=llama3_instruct, summarizer=summarizer),
    FunctionCallTool(model=nexus_raven, fn=create_event),
    FunctionCallTool(model=nexus_raven, fn=engine.search_images),
    RespondTool(model=llama3_instruct),
]

client = ChatClient(model=mistral_finetuned)
planner = FineTunedPlanner(client=client)
agent = Agent(planner=planner, tools=tools, ask_user=False)

/home/glados/miniconda3/envs/grammar-based-agents/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
agent.run("what is Leo DiCaprio's current girlfriend's age raised to the 0.24 power?")

Task: Search for Leo DiCaprio's current girlfriend's name and age.


2024-05-24 18:38:13,679 [1091672] [gba.tools.search.search_internet] [WARNING]: Searching the internet for query 'What is Leo DiCaprio's current girlfriend's name and age?'


Observation: Leo DiCaprio's current girlfriend's name is Vittoria Ceretti, and her age is 25.

Task: Calculate the value of 25 raised to the power of 0.24 using a mathematical formula or a calculator.
```python
girlfriend_age = 25
exponent = 0.24
result = pow(girlfriend_age, exponent)
```
Observation: The value of 25 raised to the power of 0.24 is 2.165237809112449.



"According to the available information, Leo DiCaprio's current girlfriend Vittoria Ceretti is 25 years old, and when her age is raised to the power of 0.24, it equals approximately 2.165237809112449."

In [8]:
agent.run("How many years did sharks appear earlier on Earth, compared to octopuses?")

Task: Search Wikipedia for the estimated year sharks first appeared on Earth.


2024-05-24 18:38:48,052 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'What is the estimated year sharks first appeared on Earth?'


Observation: According to the context, sharks first appeared on Earth around 450 million years ago during the Late Ordovician Period.

Task: Search Wikipedia for the estimated year octopuses first appeared on Earth.


2024-05-24 18:39:00,529 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'What is the estimated year octopuses first appeared on Earth?'


Observation: The estimated year octopuses first appeared on Earth is over 470 million years ago.

Task: Calculate the difference between 470 million years and 450 million years.
```python
shark_appearance_year = 450 * 10 ** 6
octopus_appearance_year = 470 * 10 ** 6
result = octopus_appearance_year - shark_appearance_year
```
Observation: The difference between 470 million years and 450 million years is 20,000,000.



'Sharks appeared on Earth around 450 million years ago, while octopuses first appeared over 470 million years ago, which means that octopuses appeared earlier by approximately 20 million years.'

In [9]:
agent.run("How many years did sharks appear earlier on Earth, compared to elephants?")

Task: Search Wikipedia for the age of sharks on Earth.


2024-05-24 18:39:31,992 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'What is the age of sharks on Earth?'


Observation: Sharks have inhabited our oceans for more than 500 million years, pre-dating the first dinosaurs by 100 million years.

Task: Search Wikipedia for the age of elephants on Earth.


2024-05-24 18:39:46,602 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'What is the average age of elephants on Earth?'


Observation: Elephants reach adulthood at 17 years of age in both sexes.

Task: Search Wikipedia for the age of elephants on Earth.


2024-05-24 18:39:59,104 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'What is the average age of elephants on Earth?'


Observation: Elephants reach adulthood at 17 years of age in both sexes.

Task: Search Wikipedia for the age of elephants on Earth, focusing on their appearance rather than adulthood age.


2024-05-24 18:40:12,009 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'What is the average age of elephants on Earth when they reach their physical maturity?'


Observation: Elephants attain their maximum stature when they complete the fusion of long-bone epiphyses, occurring in males around the age of 40 and females around 25 years of age.

Task: Calculate the difference in years between 500 million years for sharks and the maximum stature age of elephants, which is around 40 years for males and 25 years for females.
```python
shark_age = 500000000
male_elephant_age = 40
female_elephant_age = 25
result = shark_age - male_elephant_age
```
Observation: The difference in years between 500 million years for sharks and the maximum stature age of elephants is approximately 499,999,960 years.



'Sharks appeared on Earth more than 499,999,960 years earlier than elephants, with sharks inhabiting the oceans for over 500 million years and elephants reaching their maximum stature at around 40 years of age in males.'

In [10]:
agent.run("When did the first episode of the Simpsons air and how old is the series now?")

Task: Search Wikipedia for the air date of the first episode of The Simpsons.


2024-05-24 18:40:52,207 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'What is the air date of the first episode of The Simpsons?'


Observation: The air date of the first episode of The Simpsons is December 17, 1989.

Task: Calculate the age of The Simpsons by subtracting 1989 from 2023.
```python
air_date_year = 1989
current_year = 2023
result = current_year - air_date_year
```
Observation: The Simpsons is 34 years old.



'The first episode of The Simpsons aired on December 17, 1989, and the series is currently 34 years old.'

In [11]:
agent.run("How many years are between the release of Witcher 3 and Cyberpunk 2077?")

Task: Search for the release date of The Witcher 3.


2024-05-24 18:41:21,463 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'What is the release date of The Witcher 3?'


Observation: The Witcher 3: Wild Hunt was released worldwide on 19 May 2015.

Task: Search for the release date of Cyberpunk 2077.


2024-05-24 18:41:33,952 [1091672] [gba.tools.search.search_internet] [WARNING]: Searching the internet for query 'What is the release date of Cyberpunk 2077?'


Observation: The release date of Cyberpunk 2077 is December 9, 2020.

Task: Calculate the number of years between December 9, 2020, and May 19, 2015.
```python
release_date_cyberpunk = 2020
release_date_witcher = 2015
result = release_date_cyberpunk - release_date_witcher
```
Observation: There are 5 years between December 9, 2020, and May 19, 2015.



"There are 5 years between the release of The Witcher 3: Wild Hunt in May 2015 and Cyberpunk 2077's release on December 9, 2020."

In [12]:
agent.run("Who directed the movie Jurassic Park and which other movies did this person direct?")

Task: Search Wikipedia for the director of Jurassic Park.


2024-05-24 18:42:11,691 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'director of Jurassic Park'


Observation: The director of Jurassic Park is Steven Spielberg.

Task: Search Wikipedia for a list of movies directed by Steven Spielberg.


2024-05-24 18:42:21,351 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'Steven Spielberg movies directed'


Observation: Steven Spielberg directed films such as Jaws (1975), Close Encounters of the Third Kind (1977), E.T. the Extra-Terrestrial (1982), Jurassic Park (1993), Indiana Jones series, Poltergeist (1982 film), and many others mentioned in the context.



'The director of Jurassic Park is Steven Spielberg, who also directed other notable movies such as Jaws, Close Encounters of the Third Kind, E.T. the Extra-Terrestrial, and Indiana Jones series.'

In [14]:
agent.run("Who directed the movie Saving Private Ryan and which movie from this director was the most successful according to box office numbers?")

Task: Search Wikipedia for the director of the movie Saving Private Ryan.


2024-05-24 18:44:54,873 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'Director of Saving Private Ryan'


Observation: The director of the movie Saving Private Ryan is Steven Spielberg.

Task: Search Wikipedia for a list of Steven Spielberg's movies and their box office earnings.


2024-05-24 18:45:02,514 [1091672] [gba.tools.search.search_wikipedia] [WARNING]: Searching wikipedia for query 'Steven Spielberg movies box office earnings'


Observation: No information found.

Task: Search the internet for a list of Steven Spielberg's movies and their box office earnings.


...

2024-05-24 18:46:12,524 [1091672] [gba.tools.search.search_internet] [WARNING]: Searching the internet for query 'Schindler's List box office earnings'


Observation: The box office earnings of Schindler's List were $96,898,818 domestically and 322 million internationally.

Task: Search the internet for a list of all Steven Spielberg's movies and their box office earnings to confirm if there are any other movies besides Jurassic Park, E.T., and Schindler's List that had higher box office earnings.


2024-05-24 18:46:26,835 [1091672] [gba.tools.search.search_internet] [WARNING]: Searching the internet for query 'What are Steven Spielberg's movies and their box office earnings?'


Observation: According to the context, Steven Spielberg's highest-grossing movies are "The Lost World: Jurassic Park" with $618,638,999, "Raiders of the Lost Ark" with $389,925,971, "Jurassic Park" with $1,109,832,231, and "Saving Private Ryan" with $482,349,603.



'Steven Spielberg directed the movie Saving Private Ryan, and his most successful movie according to box office numbers is Jurassic Park with a total of $1.114 billion.'

In [6]:
agent.run("Add Martin's birthday party, starting Dec. 17th 2023 8pm, "
          "to my calendar and respond with a one-line invitation")

Task: Create an event titled 'Martin's Birthday Party' starting on December 17th at 8pm in 2023.
Call: create_event(title='Martin\'s Birthday Party', date='12/17/2023', time='8pm')
Observation: Event Martin's Birthday Party successfully added to calendar, date=12/17/2023, time=8pm



"You're invited to Martin's Birthday Party on December 17th, 2023 at 8pm!"

In [7]:
agent.run("What are the 3 primary colors?")

'The 3 primary colors are red, yellow, and blue.'